In [1]:
import pandas as pd
import psycopg2

In [2]:
conexion = psycopg2.connect(
    host="mikldb.c4q77ebplhtf.us-east-2.rds.amazonaws.com", 
    database="postgres", 
    user="postgres", 
    password="*Mexico2022*"
)
cur = conexion.cursor()

In [3]:
alive_cows = pd.read_csv('../Resources/alive_cows_to_predict.csv')
ids_cows = alive_cows[['id']]
list_of_single_column = ids_cows['id'].tolist()
string_ids = ids_cows['id'].astype(str).tolist()
string_ids = ','.join(string_ids)

In [4]:
cur.execute(f"""select * from milk_lactation_summary where id_cow in ({string_ids});""")
alive_cows_raw = cur.fetchall()
alive_cows_raw = pd.DataFrame(alive_cows_raw, columns=['id_cow', 'lactation', 'days_milk', 'daily_production', 'id_lac'])
alive_cows_raw = alive_cows_raw[['id_cow', 'lactation', 'days_milk', 'daily_production']]
df = alive_cows_raw
for i in range(1,10):
    df['total_production_' + str(i)] = 0
for id in list_of_single_column:
    filter_df = df.query(f'id_cow == {id}')
    if filter_df.lactation.unique().any():
        for lact in list(filter_df.lactation.unique()):
            index_ = df.query(f'id_cow == {id}').index[0]
            index_t = df.query(f'id_cow == {id}').index[lact - 1]
            df.iloc[index_]['total_production_' + str(lact)] = df.iloc[index_t]['daily_production']

In [5]:
for i in range(1,10):
    df['days_milk' + str(i)] = 0
for id in list_of_single_column:
    filter_df = df.query(f'id_cow == {id}')
    if filter_df.lactation.unique().any():
        for lact in list(filter_df.lactation.unique()):
            index_ = df.query(f'id_cow == {id}').index[0]
            index_t = df.query(f'id_cow == {id}').index[lact - 1]
            df.iloc[index_]['days_milk' + str(lact)] = df.iloc[index_t]['days_milk']

In [6]:
for i in range(1,10):
    df['lactation_' + str(i)] = 0
for id in list_of_single_column:
    filter_df = df.query(f'id_cow == {id}')
    if filter_df.lactation.unique().any():
        for lact in list(filter_df.lactation.unique()):
            index_ = df.query(f'id_cow == {id}').index[0]
            df.iloc[index_]['lactation_' + str(lact)] = 1

In [7]:
group = df.groupby('id_cow').first()
group_df = group.reset_index()

In [8]:
group_df = group_df.drop(['lactation', 'daily_production', 'days_milk'], axis=1)
cur.execute(f"""select * from milk_abortion_details where id_cow in ({string_ids});""")
abort_cows = cur.fetchall()
abort_cows = pd.DataFrame(abort_cows, columns=['id_cow', 'date', 'ab_1', 'ab_2', 'ab_3', 'ab_4', 'ab_5', 'ab_6', 'ab_7', 'ab_8', 'ab_9'])
abort_cows = abort_cows[['id_cow', 'ab_1', 'ab_2', 'ab_3', 'ab_4', 'ab_5', 'ab_6', 'ab_7', 'ab_8', 'ab_9']]

cur.execute(f"""select * from milk_insemination_details where id_cow in ({string_ids});""")
inse_cows = cur.fetchall()
inse_cows = pd.DataFrame(inse_cows, columns=['id_cow', 'date', 'inse_1', 'inse_2', 'inse_3', 'inse_4', 'inse_5', 'inse_6', 'inse_7', 'inse_8', 'inse_9'])
inse_cows = inse_cows[['id_cow','inse_1', 'inse_2', 'inse_3', 'inse_4', 'inse_5', 'inse_6', 'inse_7', 'inse_8', 'inse_9']]

cur.execute(f"""select * from milk_mastitis_details where id_cow in ({string_ids});""")
mast_cows = cur.fetchall()
mast_cows = pd.DataFrame(mast_cows, columns=['id_cow','date','mast_1', 'mast_2', 'mast_3', 'mast_4', 'mast_5', 'mast_6', 'mast_7', 'mast_8', 'mast_9'])
mast_cows = mast_cows[['id_cow','mast_1', 'mast_2', 'mast_3', 'mast_4', 'mast_5', 'mast_6', 'mast_7', 'mast_8', 'mast_9']]

In [9]:
total_dt = pd.merge(group_df, abort_cows, how='left', on='id_cow').fillna(False, downcast='infer')
total_dt = pd.merge(total_dt, inse_cows, how='left', on='id_cow').fillna(False, downcast='infer')
total_dt = pd.merge(total_dt, mast_cows, how='left', on='id_cow').fillna(False, downcast='infer')
total_dt.columns

Index(['id_cow', 'total_production_1', 'total_production_2',
       'total_production_3', 'total_production_4', 'total_production_5',
       'total_production_6', 'total_production_7', 'total_production_8',
       'total_production_9', 'days_milk1', 'days_milk2', 'days_milk3',
       'days_milk4', 'days_milk5', 'days_milk6', 'days_milk7', 'days_milk8',
       'days_milk9', 'lactation_1', 'lactation_2', 'lactation_3',
       'lactation_4', 'lactation_5', 'lactation_6', 'lactation_7',
       'lactation_8', 'lactation_9', 'ab_1', 'ab_2', 'ab_3', 'ab_4', 'ab_5',
       'ab_6', 'ab_7', 'ab_8', 'ab_9', 'inse_1', 'inse_2', 'inse_3', 'inse_4',
       'inse_5', 'inse_6', 'inse_7', 'inse_8', 'inse_9', 'mast_1', 'mast_2',
       'mast_3', 'mast_4', 'mast_5', 'mast_6', 'mast_7', 'mast_8', 'mast_9'],
      dtype='object')

In [10]:
total_dt.fillna(0)
total_dt

,id_cow,total_production_1,total_production_2,total_production_3,total_production_4,total_production_5,total_production_6,total_production_7,total_production_8,total_production_9,...,inse_9,mast_1,mast_2,mast_3,mast_4,mast_5,mast_6,mast_7,mast_8,mast_9
0,5,7817,9123,9129,8432,0,0,0,0,0,...,-1,0,0,0,0,-1,-1,-1,-1,-1
1,9,7793,10019,7907,8079,0,0,0,0,0,...,-1,0,0,0,0,-1,-1,-1,-1,-1
2,16,10363,10293,2655,10402,0,0,0,0,0,...,-1,0,0,0,0,-1,-1,-1,-1,-1
3,27,9367,11250,2090,0,0,0,0,0,0,...,-1,0,0,0,-1,-1,-1,-1,-1,-1
4,121,11491,12331,12403,12893,11326,7259,13318,0,0,...,-1,0,0,0,0,1,0,0,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4388,20423,10430,0,0,0,0,0,0,0,0,...,-1,1,-1,-1,-1,-1,-1,-1,-1,-1
4389,20426,7725,0,0,0,0,0,0,0,0,...,-1,0,-1,-1,-1,-1,-1,-1,-1,-1
4390,20428,3732,0,0,0,0,0,0,0,0,...,-1,0,-1,-1,-1,-1,-1,-1,-1,-1
4391,20429,1375,0,0,0,0,0,0,0,0,...,-1,0,-1,-1,-1,-1,-1,-1,-1,-1


In [11]:
total_dt['production_ok'] = 0
total_dt.loc[total_dt['total_production_3'] > 10000, 'production_ok'] = 1
# obtener solo las vacas que tienen dos lactancias
total_dt = total_dt.query('lactation_2 != 0')

In [14]:
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

In [15]:
df_new = total_dt[[
     'lactation_1', 'lactation_2',  'total_production_1', 'total_production_2', 'days_milk1', 
    'days_milk2', 'mast_1',
       'mast_2',
       'inse_1', 'inse_2',
       'ab_1', 'ab_2',
    'production_ok'
]]

In [16]:
# Define the features set.
X = df_new.copy()
X = X.drop("production_ok", axis=1)
X.head()

,lactation_1,lactation_2,total_production_1,total_production_2,days_milk1,days_milk2,mast_1,mast_2,inse_1,inse_2,ab_1,ab_2
0,1,1,7817,9123,288,319,0,0,1,3,0,0
1,1,1,7793,10019,290,322,0,0,1,3,0,0
2,1,1,10363,10293,310,392,0,0,2,2,0,0
3,1,1,9367,11250,334,403,0,0,2,5,0,0
4,1,1,11491,12331,343,301,0,0,4,2,0,0


In [18]:
# Define the target set.
y = df_new["production_ok"].ravel()
y[:5]

array([0, 0, 0, 0, 1], dtype=int64)

In [19]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [20]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

In [21]:
# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [23]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [25]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [26]:
predictions

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

In [27]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

In [28]:
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,457,37
Actual 1,114,64


In [29]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [30]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,457,37
Actual 1,114,64


Accuracy Score : 0.7752976190476191
Classification Report
              precision    recall  f1-score   support

           0       0.80      0.93      0.86       494
           1       0.63      0.36      0.46       178

    accuracy                           0.78       672
   macro avg       0.72      0.64      0.66       672
weighted avg       0.76      0.78      0.75       672



In [31]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.        , 0.        , 0.18591541, 0.22558564, 0.16827408,
       0.17705354, 0.02000262, 0.03215179, 0.06901342, 0.08764146,
       0.01922636, 0.01513568])

In [32]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.22558564413146973, 'total_production_2'),
 (0.1859154067086506, 'total_production_1'),
 (0.1770535381199262, 'days_milk2'),
 (0.16827407683852647, 'days_milk1'),
 (0.08764145723089427, 'inse_2'),
 (0.06901342487995042, 'inse_1'),
 (0.032151789106898455, 'mast_2'),
 (0.020002622732834183, 'mast_1'),
 (0.019226357370536025, 'ab_1'),
 (0.01513568288031379, 'ab_2'),
 (0.0, 'lactation_2'),
 (0.0, 'lactation_1')]

In [33]:
# Modelanco cambiando el valor n_estimators=128 a n_estimators =500

In [34]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

In [35]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [36]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [37]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,

In [38]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

In [39]:
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,455,39
Actual 1,113,65


In [40]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [41]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,455,39
Actual 1,113,65


Accuracy Score : 0.7738095238095238
Classification Report
              precision    recall  f1-score   support

           0       0.80      0.92      0.86       494
           1       0.62      0.37      0.46       178

    accuracy                           0.77       672
   macro avg       0.71      0.64      0.66       672
weighted avg       0.75      0.77      0.75       672



In [42]:
importances = rf_model.feature_importances_
importances

array([0.        , 0.        , 0.18448882, 0.22442837, 0.17000587,
       0.17550184, 0.02038959, 0.03177602, 0.07063785, 0.0884858 ,
       0.01942948, 0.01485635])

In [43]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.2244283691626824, 'total_production_2'),
 (0.18448881854200447, 'total_production_1'),
 (0.1755018443138867, 'days_milk2'),
 (0.17000587427554747, 'days_milk1'),
 (0.08848580105979946, 'inse_2'),
 (0.07063785156551705, 'inse_1'),
 (0.03177602493389459, 'mast_2'),
 (0.020389591936122994, 'mast_1'),
 (0.019429477162976983, 'ab_1'),
 (0.014856347047567685, 'ab_2'),
 (0.0, 'lactation_2'),
 (0.0, 'lactation_1')]